# Tips for cost-effective usage of Azure Cosmos DB at any scale

In this notebook, we'll learn how to scale Azure Cosmos containers manually and using autoscale.

To run this C# notebook, be sure to select the **CSharp** kernel in the command bar, so we can get all the language support features we need.


## Create database and containers
First, we'll create a new database and container to hold our data. Note when we create the container, we select a value for the partition key: we'll partition our data by the item id value, as it has a high cardinality (important for workloads during a lot of writes, e.g. IOT workloads) and evenly distributes the request and storage volume. Choosing a good partition key is "key" to getting good scale and performance from Azure Cosmos DB, so it's important we follow the [best practices](https://docs.microsoft.com/azure/cosmos-db/partitioning-overview)!

### To start, we'll create a container that has the minimum 400 RU/s of throughput.

In [1]:
using System;
using Microsoft.Azure.Cosmos;
using System.Collections;
using System.Collections.Generic;

// Initialize a new instance of CosmosClient using the built-in account endpoint and key parameters
CosmosClient cosmosClient = new CosmosClient(Cosmos.Endpoint, Cosmos.Key);

// Create a new database and a new container
Microsoft.Azure.Cosmos.Database database = await cosmosClient.CreateDatabaseIfNotExistsAsync("WebsiteData");

ContainerProperties containerProperties = new ContainerProperties()
{
    Id = "scale-test",
    PartitionKeyPath = "/id",
    IndexingPolicy = new IndexingPolicy()
   {
        Automatic = false,
        IndexingMode = IndexingMode.None,
   }
};

Container container = await database.CreateContainerIfNotExistsAsync(containerProperties, throughput: 400);

Display.AsMarkdown(@"
Created database WebsiteData and container Sales. You can see these new resources by refreshing your resource pane under the Data section.
");


Created database WebsiteData and container Sales. You can see these new resources by refreshing your resource pane under the Data section.


### Ingesting data

We'll ingest an example retail dataset with a similar structure to the retail dataset from the previous example. This retail dataset has approx. 2,600 documents that are 1 KB each. Let's estimate how long it will take to ingest 2,600 with our 400 RU/s of provisioned throughput.

Since each document is 1 KB, it will cost about 5 RUs of throughput to ingest each document. Since we have 400 RU/s of provisioned throughput, we have 400 RUs of throughput available each second. Therefore, we can ingest approximately 80 documents per second.

With a ingestion rate of 80 documents per second, it should take approx. 30 seconds to ingest our full dataset of 2,600 documents

In [3]:
%%upload --databaseName WebsiteData --containerName scale-test --url https://cosmosnotebooksdata.blob.core.windows.net/notebookdata/websiteData.json

Documents successfully uploaded to scale-test 
Total number of documents imported: 
  Success: 2654 
  Failure: 0 
Total time taken : 00:00:30 hours 
Total RUs consumed : 13137.300000000514

### Scaling up throughput

For workloads with small documents, 400 RU/s allows for 80 writes/sec. While this is sufficient scale in many cases, some customers need a higher provisioned throughput. In the next code sample, we'll scale up our scale-test container to 4,000 RU/s, allowing data ingestion at 10x the rate of 400 RU/s

When scaling Azure Cosmos containers, it's good to keep in mind that:

- For most changes that are less than a 10x increase, containers scale instantly. In rare cases that require a substantial increase in scale, scaling takes a few hours.
- Scaling a container (either up or down) will never cause downtime. 
- You can scale a container programmatically (uses SDKs or Azure CLI) as well as in the Azure Portal

Let's try it out!